# Model 1

## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [ ]:
import os
from pathlib import Path
import pandas as pd
import sys

import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

# Get the absolute path to the utils directory (it's in the root, not src/utils)
utils_path = Path(r"C:\Users\mhu\Documents\github\topic_model_it\utils")
if str(utils_path) not in sys.path:
    sys.path.insert(0, str(utils_path))


# Now import the module
from utils import calculate_sample_size, label_courses_with_gpt


In [8]:
# CSV laden
# Workspace root used throughout the notebook
target = Path("C:/Users/mhu/Documents/github/topic_model_it")
ai_labeled_path = target / "data" / "ai_labeld_data.csv"
df_ai = pd.read_csv(ai_labeled_path)

# Filter hat_text == True und 100 zufällige Zeilen auswählen
human_rows = df_ai[df_ai["hat_text"] == True].sample(n=100, random_state=globals().get("seed", None)).copy()

# Initialisiere die Spalte im Original-DF mit False
df_ai["human_rows"] = False

# Setze True für alle Indizes, die in deiner Stichprobe vorkommen
df_ai.loc[human_rows.index, "human_rows"] = True

In [ ]:
df_ai

## Kalkulation Samplesize

Wie berechnen die Samplesize. 

In [2]:
from pathlib import Path
import os

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

topic_model_1 = pd.read_csv("data/informatikkurse.csv") 

len_data = len(topic_model_1)
print(len_data)
result = calculate_sample_size(N=len_data, p=0.5, z=1.96, e=0.03)
print(f"Benötigte Samples zum Labeln: {result}")

51248
Benötigte Samples zum Labeln: 1046


C:\Users\mhu\AppData\Local\Temp\ipykernel_30428\936324303.py:7: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  topic_model_1 = pd.read_csv("data/informatikkurse.csv")


Wir exportieren 1046 Zeilen für testlabeling:

In [ ]:
# Spaltennamen laut deinem Screenshot
cols = ['veranstaltung_titel', 'kursbeschreibung']

# Sample ziehen
sampled = topic_model_1.drop_duplicates(subset=cols).sample(n=result, random_state=globals().get('seed', None))[cols].copy()

# Index als ID behalten, damit du es später mergen kannst
sampled.insert(0, 'original_id', sampled.index)

# Hilfsspalte: Hat der Kurs überhaupt eine Beschreibung? 
# Das hilft dir, die 100 Zeilen für dich selbst auszuwählen (nimm lieber die mit Text!)
sampled['hat_text'] = sampled['kursbeschreibung'].notna()

# Leere Spalten für das Labeling
sampled['Label_ChatGPT'] = ""
sampled['Label_Mensch'] = ""
sampled['Begründung_KI'] = ""

In [4]:
my_topics_string = """
Computer Science Education
Scientific Work Education
Deep Learning and AI
Natural Language Processing
Security and Privacy
Cryptography
Computer Science Introduction
Software Engineering
Operating Systems
Computer Architecture and Systems
Distributed Systems and Cloud Computing
Computer Networking
Database Systems
Algorithms and Data Structures
Theoretical Computer Science
Graph Theory and Linear Algebra
Human-Computer Interaction
Virtual and Augmented Reality
Media Informatics
Indie Game Development
Robotics
Autonomous Vehicles
Cyber-Physical Systems
Bioinformatics
Medical Image Processing
Energy Informatics
Informatik Kolloquium
Sonstiges
"""

In [7]:
test_df = sampled.sample(n=25, random_state=42)

labeled_df, all_topics, new_topics = label_courses_with_gpt(df=sampled,
                                                            topic_list_str = my_topics_string,
                                                            output_path = target / "data" / "ai_labeld_data.csv")

Starte dynamisches Informatik-Labeling. Basis-Topics: 28
Verarbeite Kurs 1/1046: Simultaneous Localization and Mapping (SLAM)
Verarbeite Kurs 2/1046: Implementierung Relationaler Datenbanksysteme (DB 2), früher Datenbanksysteme II
Verarbeite Kurs 3/1046: Programmierung B (Übung)
Verarbeite Kurs 4/1046: Computer und Ethik
Verarbeite Kurs 5/1046: Phylogeny and Evolution
Verarbeite Kurs 6/1046: Vorlesung: 2.01.661 Digitale Transformation
Verarbeite Kurs 7/1046:  Praktikum Systemadministration (IN0012, IN2106, IN4135) 
Verarbeite Kurs 8/1046: Kolloquium Bildschirmtechnik
Verarbeite Kurs 9/1046: Introduction to Machine Learning
									      				 - 4 SWS  
 - englisch
Verarbeite Kurs 10/1046: Inf-MSc-SemOR: Masterseminar - Operations Research/Effiziente Algorithmen
Verarbeite Kurs 11/1046: Übung Weiterführende Bioinformatik (IN2399)
Verarbeite Kurs 12/1046: Einführung in die Künstliche Intelligenz (Übung) (LV / Übung)
Verarbeite Kurs 13/1046: Seminar Theoretische Informatik
Verarbeite Kurs

In [ ]:
# labeled_df

,original_id,veranstaltung_titel,kursbeschreibung,hat_text,Label_ChatGPT,Label_Mensch,Begründung_KI
38403,38403,Mathematische Methoden der Signalverarbeitung,Mathematische Methoden der Signalverarbeitung\...,True,Theoretical Computer Science,,Der Kurs behandelt mathematische Methoden und ...
27175,27175,Practical parallel algorithms with MPI,Introduction to parallel programming and HPC s...,True,Distributed Systems and Cloud Computing,,Der Kurs behandelt parallele Programmierung un...
42057,42057,VIS-Proj-M: Masterprojekt Informationsvisualis...,"In the project, students explore and apply dif...",True,Natural Language Processing,,Der Kurs fokussiert sich auf die Anwendung von...
39416,39416,Networks and Communication Systems,DRINGEND BEACHTEN: Um die erforderlichen Passw...,True,Computer Networking,,Der Kurs behandelt Netzwerke und Kommunikation...
48741,48741,Deep Learning 1 (Klausureinsicht),NaN,False,Deep Learning and AI,,Der Kurs behandelt spezifisch Aspekte des Deep...
25376,25376,Seminar Computer Vision,Seminar Computer Vision (CV) will consist most...,True,Computer Vision,,Der Kurs fokussiert sich auf die wissenschaftl...
25191,25191,Rechnerübungen zu Betriebssystemtechnik,NaN,False,Operating Systems,,Der Kurs behandelt spezifische Übungen zur Bet...
19975,19975,Seminar: Wahlfach Einführung in die Versorgung...,Hintergrund Die Versorgungsforschung ist ein i...,True,Sonstiges / Keine Zuordnung möglich,,"Der Kurs behandelt Versorgungsforschung, die n..."
28710,28710,CPS-Forschungsprojekt Bio-Signalverarbeitung f...,Medizinische Exoskelette sind ein wichtiges Th...,True,Bioinformatics,,Das Projekt befasst sich mit der Analyse von B...
9468,9468,Einführung in die Methoden der künstlichen Int...,Weitere Informationen unter: http://www.ki.inf...,True,Deep Learning and AI,,Der Kurs behandelt zentrale Methoden und Frage...


## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [ ]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [ ]:
# Anzeige der Spaltennamen von data_model_1
print(data_model_1.columns.tolist())

data_model_1 = data_model_1[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_1.head()

data_model_1 = data_model_1.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_1['titel_kursbesch'] = data_model_1['veranstaltung_titel'] + ' ' + data_model_1['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_1['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

## Stopwords

In [ ]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

# Model

Wir definieren unser Model.

In [ ]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

## Model-Settings, GPU-Check
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

Wir checken weiterhin, ob wirklich unsere GPU verwendet wird. Wenn `cuda:0` ist dies der Fall.

In [ ]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

# BERTopic initialisieren
topic_model = BERTopic(
  embedding_model=embedding_model,
  #min_topic_size=10,
  nr_topics=20, 
  language="multilingual",
  verbose=True,
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 15,
  representation_model=representation_model
)

## Training

Wir trainieren das Modell.

In [ ]:
topic_model_1 = topic_model.fit(docs)
topic_model_1.get_topic_info()

## Outlier Reduzieren, Topics mergen

In [ ]:
# topic_model.merge_topics(docs, [0, 13])

# Outlier reduzieren
# BERTopic auf Test-Daten anwenden
# topics, probs = topic_model_1.transform(docs)
# print(topic_model_1.get_topic_freq())

# # Outlier reduzieren
# topics = topic_model_1.reduce_outliers(docs, topics)

In [ ]:
topic_model_1.get_topic_info()

## Labeling

Mit Hilfe von gpt4o lassen wir uns die Topcis labeln. Das erleichtert die Interpretation des Modells

In [ ]:
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(
    client=client, 
    model="gpt-4o-mini", 
    delay_in_seconds=10,      # Erhöhe Delay
    exponential_backoff=True, # Aktiviere das schrittweise längere Warten
    nr_docs=3                 # Weniger Dokumente pro Topic reduzieren die Token-Last
)
topic_model.update_topics(docs, representation_model=rep)
topic_model.get_topic_info() 

In [ ]:
topic_model.get_topic_info() 

In [ ]:
# Speichern
topic_model.save("models/explor_model_1", serialization="safetensors", save_embedding_model=True)

# Laden
from bertopic import BERTopic
loaded_model = BERTopic.load("models/explor_model_1")